# Time Series Analysis in Python

Welcome to your webinar workspace! You can follow along as we go through an introduction to time series analysis in Python. To consult the solution, head over to the file browser and select `notebook-solution.ipynb`.

This first cell imports some of the main packages we will be using, as well as sets the visualization theme we will be using.

In [158]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from datetime import datetime

# Set colors
dc_colors = ["#2B3A64", "#96aae3", "#C3681D", "#EFBD95", "#E73F74", "#80BA5A", "#E68310", "#008695", "#CF1C90", "#f97b72", "#4b4b8f", "#A5AA99"]

# Set template
pio.templates["dc"] = go.layout.Template(
    layout=dict(
    	font={"family": "Poppins, Sans-serif", "color": "#505050"},
        title={"font": {"family": "Poppins, Sans-serif", "color": "black"}, "yanchor": "top", "y": 0.92, "xanchor": "left", "x": 0.025},
    	plot_bgcolor="white",
    	paper_bgcolor="white",
    	hoverlabel=dict(bgcolor="white"),
    	margin=dict(l=100, r=50, t=75, b=70),
        colorway=dc_colors,
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=True, 
                   gridwidth=0.1, 
                   gridcolor='lightgrey', 
                   showline=True,
                   nticks=10,
                   linewidth=1, 
                   linecolor='black', 
                   rangemode="tozero")
    )
) 

## Loading and Inspecting the Data
The first thing we will do is use the [`yfinance`](https://pypi.org/project/yfinance/) package to download market data from the Yahoo! Finance API.

We will define the date range that we want to use, as well as the ticker we want to download.

In [159]:
# Import yfinance
import yfinance as yf

# Set the date range
start = '2020-01-01'
stop = '2023-02-01'

# Set the ticker we want to use (GameStop)
ticker = "GME"

# Get the data for the ticker GME
gme = yf.download(ticker, start, stop)

# Preview DataFrame
gme

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02 00:00:00-05:00,1.535000,1.617500,1.517500,1.577500,1.577500,17814400
2020-01-03 00:00:00-05:00,1.552500,1.562500,1.460000,1.470000,1.470000,14175600
2020-01-06 00:00:00-05:00,1.450000,1.477500,1.400000,1.462500,1.462500,13579200
2020-01-07 00:00:00-05:00,1.442500,1.457500,1.360000,1.380000,1.380000,20912000
2020-01-08 00:00:00-05:00,1.372500,1.462500,1.352500,1.430000,1.430000,22517600
...,...,...,...,...,...,...
2023-01-25 00:00:00-05:00,20.590000,20.840000,19.530001,20.230000,20.230000,3515800
2023-01-26 00:00:00-05:00,20.610001,21.170000,19.379999,20.010000,20.010000,3520000
2023-01-27 00:00:00-05:00,19.799999,23.309999,19.410000,22.820000,22.820000,11868800


We can also use the `.describe()` method to get a sense of the data over the period.

In [160]:
# Get a numeric summary of the data
gme.describe()

,Open,High,Low,Close,Adj Close,Volume
count,776.000000,776.000000,776.000000,776.000000,776.000000,7.760000e+02
mean,24.562152,25.939124,23.237738,24.452030,24.452030,3.110771e+07
std,19.209501,20.480236,17.971932,18.981395,18.981395,7.076841e+07
min,0.712500,0.735000,0.642500,0.700000,0.700000,1.122700e+06
25%,2.640625,2.787500,2.506875,2.634375,2.634375,6.793800e+06
50%,26.085000,27.490000,25.133750,26.115000,26.115000,1.257080e+07
75%,39.750000,40.998749,37.886249,39.628749,39.628749,2.489090e+07
max,94.927498,120.750000,72.877502,86.877502,86.877502,7.886316e+08


## Visualizing the data
Next, we can use a [Plotly line plot](https://plotly.com/python/line-charts/) to examine the data over time.

In [161]:
# Create a Plotly figure
fig = px.line(gme,
              x=gme.index,
              y="Close",
              template="dc",
              title="GameStop Closing Price (daily)"
             )

# Show the plot
fig.show()

Let's add an annotation to make it clear when key events happened. We will cover [three key events](https://www.reuters.com/article/us-retail-trading-gamestop-timeline-idUSKBN2AI0IQ) in the timeline:
- The date that the new board was announced, and r/wallstreetbets began hyping the stock.
- The date when the trading app RobinHood restricted trading for GameStop (and some other stocks).
- An late February surge fueld by more activity on r/wallstreetbets.

_Note: due to a bug with Plotly, we need to use [`strptime()`](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior) to convert the dates to milliseconds to enable our annotations._

In [162]:
# Create a filtered DataFrame for early 2021
gme_2021 = gme["2021-01": "2021-03"]

# Create a Plotly figure
fig = px.line(gme_2021,
              x=gme_2021.index,
              y="Close",
              template="dc",
              title="Early 2021 GameStop Saga"
             )

# Define three key events
short = datetime.strptime("2021-01-11", "%Y-%m-%d").timestamp() * 1000
robin = datetime.strptime("2021-01-28", "%Y-%m-%d").timestamp() * 1000
late_feb = datetime.strptime("2021-02-23", "%Y-%m-%d").timestamp() * 1000

# Add these as lines
fig.add_vline(x=short, line_width=0.5, annotation_text="r/wallstreetbets")
fig.add_vline(x=robin, line_width=0.5, annotation_text="Halt")
fig.add_vline(x=late_feb, line_width=0.5, annotation_text="Memes")

# Show the plot
fig.show()

Alternatively, we can use a [candlestick chart](https://plotly.com/python/candlestick-charts/) to get a good sense of price action.

In [163]:
# Define the candlestick data
candlestick = go.Candlestick(
    x=gme.index,
    open=gme['Open'],
    high=gme['High'],
    low=gme['Low'],
    close=gme['Close'])

# Create a candlestick figure   
fig = go.Figure(data=[candlestick])
fig.update_layout(title='GME Prices (Candlestick chart)', 
                  template="dc")                        

# Show the plot
fig.show()

## Rolling averages

The data is quite noisy. We can also use a window function to calculate the rolling mean over a certain number of periods. In our case, we'll use the past 28 days of data. 

This also smooths out the line, and still gives day-by-day performance.

In [164]:
# Calculate the 28 day rolling mean price
gme_rolling = gme.rolling("28D").mean()

# Plot the rolling average
fig = px.line(gme_rolling,
              x=gme_rolling.index,
              y="Close",
              template="dc",
              title="GameStop Closing Price (rolling 28 day average)"
             )

# Show the plot
fig.show()

## Comparing to a benchmark
It would be nice to be able to compare the performance of GameStop against a stock market index such as the S&P 500 (an index tracking the performance of 500 large US companies).

In [165]:
# Get the data for the ticker GSPC
sp = yf.download("^GSPC", start, stop)

# Rename close columns
sp = sp.rename(columns={"Close": "S&P Close"})
gme = gme.rename(columns={"Close": "GameStop Close"})

# Concatenate the data
all_data = pd.concat([gme["GameStop Close"], 
                      sp["S&P Close"]],
                      axis=1)

# Preview the data
all_data

[*********************100%***********************]  1 of 1 completed


,GameStop Close,S&P Close
Date,,
2020-01-02 00:00:00-05:00,1.577500,3257.850098
2020-01-03 00:00:00-05:00,1.470000,3234.850098
2020-01-06 00:00:00-05:00,1.462500,3246.280029
2020-01-07 00:00:00-05:00,1.380000,3237.179932
2020-01-08 00:00:00-05:00,1.430000,3253.050049
...,...,...
2023-01-25 00:00:00-05:00,20.230000,4016.219971
2023-01-26 00:00:00-05:00,20.010000,4060.429932
2023-01-27 00:00:00-05:00,22.820000,4070.560059


As you can see, the prices are on a much different scale than GameStop. Let's normalize the prices so they start at 100. To do this, we will:
- Divide all prices by the first price in the series.
- Multiply them by 100.

All prices will then be relative to the starting point. This way, we can compare large the change is between the two time series, regardless of their starting values.

In [166]:
# Select first prices
first_prices = all_data.iloc[0]

# Create normalized_prices
normalized_prices = all_data.div(first_prices).mul(100)

# Normalized
normalized_prices

,GameStop Close,S&P Close
Date,,
2020-01-02 00:00:00-05:00,100.000000,100.000000
2020-01-03 00:00:00-05:00,93.185423,99.294013
2020-01-06 00:00:00-05:00,92.709983,99.644856
2020-01-07 00:00:00-05:00,87.480191,99.365527
2020-01-08 00:00:00-05:00,90.649760,99.852662
...,...,...
2023-01-25 00:00:00-05:00,1282.408857,123.278231
2023-01-26 00:00:00-05:00,1268.462784,124.635260
2023-01-27 00:00:00-05:00,1446.592704,124.946205


We will [`.melt()`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html) the DataFrame to make it easier to plot the two time series.

In [167]:
# Melt the DataFrame to assist with plotting
normalized_melt = normalized_prices.reset_index().melt(id_vars="Date", 
                                                       var_name="Ticker", 
                                                       value_name="Closing Price")

# Preview the newly formatted data
normalized_melt

,Date,Ticker,Closing Price
0,2020-01-02 00:00:00-05:00,GameStop Close,100.000000
1,2020-01-03 00:00:00-05:00,GameStop Close,93.185423
2,2020-01-06 00:00:00-05:00,GameStop Close,92.709983
3,2020-01-07 00:00:00-05:00,GameStop Close,87.480191
4,2020-01-08 00:00:00-05:00,GameStop Close,90.649760
...,...,...,...
1547,2023-01-25 00:00:00-05:00,S&P Close,123.278231
1548,2023-01-26 00:00:00-05:00,S&P Close,124.635260
1549,2023-01-27 00:00:00-05:00,S&P Close,124.946205
1550,2023-01-30 00:00:00-05:00,S&P Close,123.325810


In [168]:
# Create a plot of the melted data
fig = px.line(normalized_melt,
              x="Date",
              y="Closing Price",
              color="Ticker",
              template="dc",
              title="GameStop vs. S&P 500 Closing Price (normalized)"
             )

# Show the plot
fig.show()

## Plotting the Autocorrelation Function

Autocorrelation is the correlation of a time series with a lagged version of itself. Plotting it can give you an idea of how lagged periods correlate to the present period.

First, let's get some recent data from when GameStop seems to have stabilized.

In [169]:
# Get recent data for GME
gme_recent = yf.download("GME", "2023-01-01", "2023-02-21")

# Preview the data
gme_recent

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2023-01-03 00:00:00-05:00,18.639999,19.260000,17.090000,17.200001,17.200001,5135200
2023-01-04 00:00:00-05:00,17.250000,17.930000,16.900000,17.320000,17.320000,3939300
2023-01-05 00:00:00-05:00,17.059999,17.260000,15.890000,16.219999,16.219999,6066200
2023-01-06 00:00:00-05:00,16.000000,16.570000,15.410000,16.459999,16.459999,4814400
2023-01-09 00:00:00-05:00,16.650000,17.129999,16.360001,16.379999,16.379999,3522600
2023-01-10 00:00:00-05:00,16.299999,18.090000,16.250000,17.770000,17.770000,4402800
2023-01-11 00:00:00-05:00,18.190001,20.049999,17.860001,19.040001,19.040001,8405800
2023-01-12 00:00:00-05:00,19.040001,20.629999,18.340000,20.629999,20.629999,5877300
2023-01-13 00:00:00-05:00,19.879999,21.110001,19.799999,20.490000,20.490000,5480800


We will use the [`acf()`](https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.acf.html) function to generate the autocorrelation function for the most recent GameStop data.

In [170]:
# Import acf
from statsmodels.tsa.stattools import acf

# Calculate the acf array for the recent GameStop data
acf_array = acf(gme_recent["Close"], nlags=10)

# Generate a scatter plot
fig = px.scatter(acf_array, template="dc")

# Fix the range and layout
fig.update_xaxes(range=[0, 10])
fig.update_layout(showlegend=False)

# Show the plot
fig.show()

First we need to fix the index before making forecasts.

In [171]:
# Set the index to the correct period
gme_recent.index = pd.DatetimeIndex(gme_recent.index).to_period('B')

# Set a new date index to handle the gaps
new_index = pd.period_range(gme_recent.index[0], 
                            gme_recent.index[-1])

gme_recent = gme_recent.reindex(new_index)

## Making Simple Forecasts
Finally, we are going to fit a model to the GameStop data up until the first of February and make a forecast. We are going to use an AR(1) model. 

$\large \quad \quad \quad \quad R\_t \quad \ \ = \quad \mu \quad + \quad \phi \quad R\_{t-1} \quad  \ + \quad \epsilon\_t$

An AR(1) model calculates the current value as a mean plus a fraction ( $ \phi $ ) of yesterday's value and some noise.
- If $ \phi $ is 0 then the process is just noise.
- If $ \phi $ is 1 then the process is a random walk.

In [172]:
# Import the ARIMA class
from statsmodels.tsa.arima.model import ARIMA

# Fit an AR(1) model to the data
mod = ARIMA(gme_recent["Close"], order=(1, 0, 0))
res = mod.fit()

# Print the model summary
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                  Close   No. Observations:                   34
Model:                 ARIMA(1, 0, 0)   Log Likelihood                 -52.228
Date:                Tue, 21 Feb 2023   AIC                            110.455
Time:                        09:17:05   BIC                            115.035
Sample:                    01-03-2023   HQIC                           112.017
                         - 02-17-2023                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         20.1466      0.882     22.838      0.000      18.418      21.876
ar.L1          0.8186      0.107      7.668      0.000       0.609       1.028
sigma2         1.3209      0.362      3.649      0.000       0.611       2.030
===================================================================================
Ljung-Box (L1) (Q):                   0.20   Jarque-Bera (JB):                 0.54
Prob(Q):                              0.65   Prob(JB):                         0.76
Heteroskedasticity (H):               1.27   Skew:                             0.26
Prob(H) (two-sided):                  0.70   Kurtosis:                         2.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Comparing different models
We ran the model with one lagged parameter. But how does our model compare to one with a different order? We can use the Akaike Information Criterion (AIC) and the Bayesian Information Criterion (BIC) to compare goodness of fit for different orders.

In [173]:
# Initialize an empty array
bic = []

# Loop through a range of AR models and get the BIC
for i in range(0, 10):
    mod = ARIMA(gme_recent["Close"], order=(i, 0, 0))
    res = mod.fit()
    bic.append(res.bic)
    
# Plot the BIC
fig = px.line(bic, template="dc")

fig.update_yaxes(rangemode="normal")

fig.show()

It looks like the lowest BIC occurs at lag 1. We can now use the [`get_forecast`](https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima.model.ARIMAResults.get_forecast.html) method to make estimates out of sample (i.e., past the range of our data).

In [182]:
# Get data up until a week ago
gme_recent_trunc = gme_recent[:"2023-02-09"]

# Estimate an AR(1) model
mod = ARIMA(gme_recent_trunc["Close"], order=(1, 0, 0))
res = mod.fit()

# Create the forecasts as a DataFrame
preds = res.get_forecast(steps=7).summary_frame()

# View the forecasts
preds

Close,mean,mean_se,mean_ci_lower,mean_ci_upper
2023-02-10,19.691563,1.204579,17.330631,22.052494
2023-02-13,19.709093,1.552465,16.666319,22.751868
2023-02-14,19.723346,1.744748,16.303702,23.142991
2023-02-15,19.734935,1.860975,16.087490,23.382379
2023-02-16,19.744356,1.933972,15.953840,23.534872
2023-02-17,19.752016,1.980748,15.869822,23.634210
2023-02-20,19.758244,2.011070,15.816620,23.699867


## Bonus: Plot the forecast
Finally, we can create a Plotly chart to visualize the forecasts with the confidence intervals.

In [183]:
# Create a figure containing predicted, real, and CI values
fig = go.Figure([
    go.Scatter(
        name='True value',
        x=gme_recent.index.to_timestamp(),
        y=gme_recent["Close"],
        mode='lines'
    ),
    go.Scatter(
        name='Predicted value',
        x=preds.index.to_timestamp(),
        y=preds["mean"],
        mode='lines'
    ),
    go.Scatter(
        name='Upper',
        x=preds.index.to_timestamp(),
        y=preds["mean_ci_upper"],
        mode='lines',
        line=dict(color='lightblue', width=0)
    ),
    go.Scatter(
        name='Lower',
        x=preds.index.to_timestamp(),
        y=preds["mean_ci_lower"],
        mode='lines',
        line=dict(color='lightblue', width=0),
        fill="tonexty"
    ),
])

# Update the layout and show the plot
fig.update_layout(
    yaxis_title='Price',
    title='GameStop Price and Forecast over Time',
    showlegend=False,
    template="dc"
) 

fig.show()